In [6]:
import torch
import cv2
import numpy as np
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import glob as glob
from xml.etree import ElementTree as et
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

print(torch.__version__)

1.12.1+cu116


### Code example found on https://debuggercafe.com/custom-object-detection-using-pytorch-faster-rcnn/
### Modified to create our implementation of training the model and cropping the Optic Disc

In [38]:
BATCH_SIZE = 4 
RESIZE_TO = 512 
NUM_EPOCHS = 20 
DEVICE = torch.device('cuda') 
print(DEVICE)

TRAIN_DIR = "..\\OD_detect\\train"
VALID_DIR = "..\\OD_detect\\test"

print(TRAIN_DIR)

CLASSES = ['background','OD']
NUM_CLASSES = 2
VISUALIZE_TRANSFORMED_IMAGES = False
OUT_DIR = '../outputs'
SAVE_PLOTS_EPOCH = 2 
SAVE_MODEL_EPOCH = 2 

def collate_fn(batch):
    return tuple(zip(*batch))

def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        A.RandomRotate90(0.5),
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc',
        'label_fields': ['labels']
    })
def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc', 
        'label_fields': ['labels']
    })


class ODDataset(Dataset):
    def __init__(self, dir_path, width, height, classes, transforms=None):
        self.transforms = transforms
        self.dir_path = dir_path
        self.height = height
        self.width = width
        self.classes = classes
        self.image_paths = glob.glob(f"{self.dir_path}\\*.jpg")
        self.all_images = [image_path.split('\\')[-1] for image_path in self.image_paths]
        self.all_images = sorted(self.all_images)
    def __getitem__(self, idx):
        image_name = self.all_images[idx]
        image_path = os.path.join(self.dir_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image_resized = cv2.resize(image, (self.width, self.height))
        image_resized /= 255.0
        
        annot_filename = image_name[:-4] + '.xml'
        annot_file_path = os.path.join(self.dir_path, annot_filename)
        
        boxes = []
        labels = []
        tree = et.parse(annot_file_path)
        root = tree.getroot()
        
        image_width = image.shape[1]
        image_height = image.shape[0]
        
        for member in root.findall('object'):
            labels.append(self.classes.index(member.find('name').text))
            
            xmin = int(member.find('bndbox').find('xmin').text)
            xmax = int(member.find('bndbox').find('xmax').text)
            ymin = int(member.find('bndbox').find('ymin').text)
            ymax = int(member.find('bndbox').find('ymax').text)
            
            xmin_final = (xmin/image_width)*self.width
            xmax_final = (xmax/image_width)*self.width
            ymin_final = (ymin/image_height)*self.height
            yamx_final = (ymax/image_height)*self.height
            
            boxes.append([xmin_final, ymin_final, xmax_final, yamx_final])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        image_id = torch.tensor([idx])
        target["image_id"] = image_id
        if self.transforms:
            sample = self.transforms(image = image_resized,
                                     bboxes = target['boxes'],
                                     labels = labels)
            image_resized = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            
        return image_resized, target
    def __len__(self):
        return len(self.all_images)
    
train_dataset = ODDataset(TRAIN_DIR, RESIZE_TO, RESIZE_TO, CLASSES, get_train_transform())
valid_dataset = ODDataset(VALID_DIR, RESIZE_TO, RESIZE_TO, CLASSES, get_valid_transform())
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(valid_dataset)}\n")


cuda
..\OD_detect\train
Number of training samples: 171
Number of validation samples: 29



In [39]:
dataset = ODDataset(TRAIN_DIR, RESIZE_TO, RESIZE_TO, CLASSES)
print(f"Number of training images: {len(dataset)}")
    
def visualize_sample(image, target):
    box = target['boxes'][0]
    label = CLASSES[target['labels']]
    cv2.rectangle(
        image, 
        (int(box[0]), int(box[1])), (int(box[2]), int(box[3])),
        (0, 255, 0), 1
    )
    cv2.putText(
        image, label, (int(box[0]), int(box[1]-5)), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2
    )
    cv2.imshow('Image', image)
    cv2.waitKey(0)

# NUM_SAMPLES_TO_VISUALIZE = 5
# for i in range(NUM_SAMPLES_TO_VISUALIZE):
#     image, target = dataset[i]
#     visualize_sample(image, target)

Number of training images: 171


In [2]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
def create_model(num_classes):
    
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 
    return model

In [41]:
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import time
plt.style.use('ggplot')

class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
    def send(self, value):
        self.current_total += value
        self.iterations += 1
    
    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations
    
    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

def train(train_data_loader, model):
    print('Training')
    global train_itr
    global train_loss_list
    
    prog_bar = tqdm(train_data_loader, total=len(train_data_loader))
    
    for i, data in enumerate(prog_bar):
        optimizer.zero_grad()
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        train_loss_list.append(loss_value)
        train_loss_hist.send(loss_value)
        losses.backward()
        optimizer.step()
        train_itr += 1
    
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
    return train_loss_list

def validate(valid_data_loader, model):
    print('Validating')
    global val_itr
    global val_loss_list
    
    prog_bar = tqdm(valid_data_loader, total=len(valid_data_loader))
    
    for i, data in enumerate(prog_bar):
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        
        with torch.no_grad():
            loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        val_loss_list.append(loss_value)
        val_loss_hist.send(loss_value)
        val_itr += 1
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
    return val_loss_list

model = create_model(num_classes=NUM_CLASSES)
model = model.to(DEVICE)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
train_loss_hist = Averager()
val_loss_hist = Averager()
train_itr = 1
val_itr = 1
train_loss_list = []
val_loss_list = []
MODEL_NAME = 'model'
if VISUALIZE_TRANSFORMED_IMAGES:
    from utils import show_tranformed_image
    show_tranformed_image(train_loader)
for epoch in range(NUM_EPOCHS):
    print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")
    train_loss_hist.reset()
    val_loss_hist.reset()
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()
    start = time.time()
    train_loss = train(train_loader, model)
    val_loss = validate(valid_loader, model)
    print(f"Epoch #{epoch} train loss: {train_loss_hist.value:.3f}")   
    print(f"Epoch #{epoch} validation loss: {val_loss_hist.value:.3f}")   
    end = time.time()
    print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")
    if (epoch+1) % SAVE_MODEL_EPOCH == 0: 
        torch.save(model.state_dict(), f"{OUT_DIR}/model{epoch+1}.pth")
        print('SAVING MODEL COMPLETE...\n')

    if (epoch+1) % SAVE_PLOTS_EPOCH == 0: 
        train_ax.plot(train_loss, color='blue')
        train_ax.set_xlabel('iterations')
        train_ax.set_ylabel('train loss')
        valid_ax.plot(val_loss, color='red')
        valid_ax.set_xlabel('iterations')
        valid_ax.set_ylabel('validation loss')
        figure_1.savefig(f"{OUT_DIR}/train_loss_{epoch+1}.png")
        figure_2.savefig(f"{OUT_DIR}/valid_loss_{epoch+1}.png")
        print('SAVING PLOTS COMPLETE...')

    if (epoch+1) == NUM_EPOCHS: 
        train_ax.plot(train_loss, color='blue')
        train_ax.set_xlabel('iterations')
        train_ax.set_ylabel('train loss')
        valid_ax.plot(val_loss, color='red')
        valid_ax.set_xlabel('iterations')
        valid_ax.set_ylabel('validation loss')
        figure_1.savefig(f"{OUT_DIR}/train_loss_{epoch+1}.png")
        figure_2.savefig(f"{OUT_DIR}/valid_loss_{epoch+1}.png")
        torch.save(model.state_dict(), f"{OUT_DIR}/model{epoch+1}.pth")

    plt.close('all')

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



EPOCH 1 of 20
Training


  0%|          | 0/43 [00:00<?, ?it/s]

..\OD_detect\train\DEV00074.jpg
..\OD_detect\train\DEV00001.jpg
..\OD_detect\train\DEV00023.jpg
..\OD_detect\train\DEV00016.jpg
..\OD_detect\train\DEV00136.jpg
..\OD_detect\train\DEV00105.jpg
..\OD_detect\train\DEV00111.jpg
..\OD_detect\train\DEV00135.jpg
..\OD_detect\train\DEV00100.jpg
..\OD_detect\train\DEV00163.jpg
..\OD_detect\train\DEV00121.jpg
..\OD_detect\train\DEV00154.jpg
..\OD_detect\train\DEV00093.jpg
..\OD_detect\train\DEV00041.jpg
..\OD_detect\train\DEV00080.jpg
..\OD_detect\train\DEV00065.jpg
..\OD_detect\train\DEV00032.jpg
..\OD_detect\train\DEV00029.jpg
..\OD_detect\train\DEV00042.jpg
..\OD_detect\train\DEV00151.jpg
..\OD_detect\train\DEV00097.jpg
..\OD_detect\train\DEV00128.jpg
..\OD_detect\train\DEV00053.jpg
..\OD_detect\train\DEV00031.jpg
..\OD_detect\train\DEV00102.jpg
..\OD_detect\train\DEV00043.jpg
..\OD_detect\train\DEV00014.jpg
..\OD_detect\train\DEV00073.jpg
..\OD_detect\train\DEV00077.jpg
..\OD_detect\train\DEV00071.jpg
..\OD_detect\train\DEV00010.jpg
..\OD_de

  0%|          | 0/8 [00:00<?, ?it/s]

...

..\OD_detect\test\DEV00171.jpg
..\OD_detect\test\DEV00172.jpg
..\OD_detect\test\DEV00173.jpg
..\OD_detect\test\DEV00174.jpg
..\OD_detect\test\DEV00175.jpg
..\OD_detect\test\DEV00176.jpg
..\OD_detect\test\DEV00177.jpg
..\OD_detect\test\DEV00178.jpg
..\OD_detect\test\DEV00179.jpg
..\OD_detect\test\DEV00180.jpg
..\OD_detect\test\DEV00181.jpg
..\OD_detect\test\DEV00182.jpg
..\OD_detect\test\DEV00183.jpg
..\OD_detect\test\DEV00184.jpg
..\OD_detect\test\DEV00185.jpg
..\OD_detect\test\DEV00186.jpg
..\OD_detect\test\DEV00187.jpg
..\OD_detect\test\DEV00188.jpg
..\OD_detect\test\DEV00189.jpg
..\OD_detect\test\DEV00190.jpg
..\OD_detect\test\DEV00191.jpg
..\OD_detect\test\DEV00192.jpg
..\OD_detect\test\DEV00193.jpg
..\OD_detect\test\DEV00194.jpg
..\OD_detect\test\DEV00195.jpg
..\OD_detect\test\DEV00196.jpg
..\OD_detect\test\DEV00197.jpg
..\OD_detect\test\DEV00198.jpg
..\OD_detect\test\DEV00199.jpg
Epoch #18 train loss: 0.044
Epoch #18 validation loss: 0.055
Took 2.335 minutes for epoch 18

EPOCH 2

  0%|          | 0/43 [00:00<?, ?it/s]

..\OD_detect\train\DEV00089.jpg
..\OD_detect\train\DEV00041.jpg
..\OD_detect\train\DEV00012.jpg
..\OD_detect\train\DEV00061.jpg
..\OD_detect\train\DEV00151.jpg
..\OD_detect\train\DEV00046.jpg
..\OD_detect\train\DEV00097.jpg
..\OD_detect\train\DEV00039.jpg
..\OD_detect\train\DEV00115.jpg
..\OD_detect\train\DEV00027.jpg
..\OD_detect\train\DEV00034.jpg
..\OD_detect\train\DEV00155.jpg
..\OD_detect\train\DEV00118.jpg
..\OD_detect\train\DEV00164.jpg
..\OD_detect\train\DEV00098.jpg
..\OD_detect\train\DEV00160.jpg
..\OD_detect\train\DEV00161.jpg
..\OD_detect\train\DEV00163.jpg
..\OD_detect\train\DEV00059.jpg
..\OD_detect\train\DEV00063.jpg
..\OD_detect\train\DEV00090.jpg
..\OD_detect\train\DEV00105.jpg
..\OD_detect\train\DEV00166.jpg
..\OD_detect\train\DEV00148.jpg
..\OD_detect\train\DEV00137.jpg
..\OD_detect\train\DEV00123.jpg
..\OD_detect\train\DEV00082.jpg
..\OD_detect\train\DEV00037.jpg
..\OD_detect\train\DEV00050.jpg
..\OD_detect\train\DEV00008.jpg
..\OD_detect\train\DEV00065.jpg
..\OD_de

  0%|          | 0/8 [00:00<?, ?it/s]

..\OD_detect\test\DEV00171.jpg
..\OD_detect\test\DEV00172.jpg
..\OD_detect\test\DEV00173.jpg
..\OD_detect\test\DEV00174.jpg
..\OD_detect\test\DEV00175.jpg
..\OD_detect\test\DEV00176.jpg
..\OD_detect\test\DEV00177.jpg
..\OD_detect\test\DEV00178.jpg
..\OD_detect\test\DEV00179.jpg
..\OD_detect\test\DEV00180.jpg
..\OD_detect\test\DEV00181.jpg
..\OD_detect\test\DEV00182.jpg
..\OD_detect\test\DEV00183.jpg
..\OD_detect\test\DEV00184.jpg
..\OD_detect\test\DEV00185.jpg
..\OD_detect\test\DEV00186.jpg
..\OD_detect\test\DEV00187.jpg
..\OD_detect\test\DEV00188.jpg
..\OD_detect\test\DEV00189.jpg
..\OD_detect\test\DEV00190.jpg
..\OD_detect\test\DEV00191.jpg
..\OD_detect\test\DEV00192.jpg
..\OD_detect\test\DEV00193.jpg
..\OD_detect\test\DEV00194.jpg
..\OD_detect\test\DEV00195.jpg
..\OD_detect\test\DEV00196.jpg
..\OD_detect\test\DEV00197.jpg
..\OD_detect\test\DEV00198.jpg
..\OD_detect\test\DEV00199.jpg
Epoch #19 train loss: 0.043
Epoch #19 validation loss: 0.064
Took 2.285 minutes for epoch 19
SAVING M

In [43]:
import numpy as np
import cv2
import torch
import glob as glob
from torchvision import transforms

device = torch.device('cuda') 
model = create_model(num_classes=2).to(device)
model.load_state_dict(torch.load(
    '..\\outputs\\model18.pth', map_location=device
))
model.eval()

DIR_TEST = '..\\test_data'
test_images = glob.glob(f"{DIR_TEST}\\*")
print(f"Test instances: {len(test_images)}")
CLASSES = ['background','OD']
detection_threshold = 0.8

for i in range(len(test_images)):
    image_name = test_images[i].split('\\')[-1].split('.')[0]
    image = cv2.imread(test_images[i])
    image_to_crop = image.copy()
    orig_image = image.copy()
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    image /= 255.0
    image = np.transpose(image, (2, 0, 1)).astype(float)
    image = torch.tensor(image, dtype=torch.float).cuda()
    image = torch.unsqueeze(image, 0)
    with torch.no_grad():
        outputs = model(image)
    
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
    if len(outputs[0]['boxes']) != 0:
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        draw_boxes = boxes.copy()
        pred_classes = [CLASSES[i] for i in outputs[0]['labels'].cpu().numpy()]
        
        for j, box in enumerate(draw_boxes):
            x_width = round((int(box[3]) - int(box[1])))
            y_height = round((int(box[2]) - int(box[0])))
            image_height = orig_image.shape[1]
            image_width = orig_image.shape[0]
            print(image_height, image_width)
            x_low = int(box[1])-x_width
            print(x_low)
            if x_low < 0:
                x_low = 0
            print(x_low)
            x_high = int(box[3])+x_width
            print(x_high)
            if x_high > image_width:
                x_high = image_width
            print(x_high)
            y_low = int(box[0])-y_height
            print(y_low)
            if y_low < 0:
                y_low = 0
            print(y_low)
            y_high = int(box[2])+y_height
            print(y_high)
            if y_high > image_height:
                y_high = image_height
            print(y_high)
            cropped_image = orig_image[x_low: x_high, y_low:y_high]
        try:
            cv2.imwrite(f"..\\test_predictions_test_data_R3\\cropped_{image_name}.jpg", cropped_image,)
        except:
            print("{} FAILED".format(image_name))
    print(f"Image {i+1} done...")
    print('-'*50)
print('TEST PREDICTIONS COMPLETE')
cv2.destroyAllWindows()

Test instances: 10000
2592 1944
602
602
1598
1598
174
174
1113
1113
Image 1 done...
--------------------------------------------------
4928 3264
789
789
2394
2394
925
925
2521
2521
Image 2 done...
--------------------------------------------------
Image 3 done...
--------------------------------------------------
2124 2056
965
965
1880
1880
949
949
1828
1828
Image 4 done...
--------------------------------------------------
1956 1934
413
413
1298
1298
-92
0
781
781
Image 5 done...
--------------------------------------------------
2592 1944
309
309
1353
1353
1325
1325
2369
2369
Image 6 done...
--------------------------------------------------
1956 1934
643
643
1441
1441
117
117
966
966
Image 7 done...
--------------------------------------------------
2560 1920
216
216
1278
1278
1376
1376
2363
2363
Image 8 done...
--------------------------------------------------
3456 2304
634
634
1666
1666
788
788
1790
1790
Image 9 done...
--------------------------------------------------
2592 1944

4928 3264
727
727
2536
2536
770
770
2297
2297
Image 73 done...
--------------------------------------------------
3888 2592
616
616
1732
1732
917
917
1997
1997
Image 74 done...
--------------------------------------------------
4928 3264
867
867
2382
2382
947
947
2375
2375
Image 75 done...
--------------------------------------------------
2560 1920
278
278
1442
1442
1342
1342
2332
2332
Image 76 done...
--------------------------------------------------
2592 1944
332
332
1373
1373
78
78
981
981
Image 77 done...
--------------------------------------------------
2560 1920
72
72
1269
1269
219
219
1260
1260
Image 78 done...
--------------------------------------------------
1876 1876
393
393
1356
1356
121
121
967
967
Image 79 done...
--------------------------------------------------
1956 1934
553
553
1369
1369
113
113
842
842
Image 80 done...
--------------------------------------------------
2560 1920
407
407
1343
1343
1561
1561
2425
2425
Image 81 done...
-------------------------------

2560 1920
148
148
1495
1495
1571
1571
2687
2560
Image 144 done...
--------------------------------------------------
2592 1944
214
214
1330
1330
187
187
1315
1315
Image 145 done...
--------------------------------------------------
2592 1944
236
236
1529
1529
1279
1279
2440
2440
Image 146 done...
--------------------------------------------------
4272 2848
769
769
2353
2353
215
215
1820
1820
Image 147 done...
--------------------------------------------------
3456 2304
559
559
1813
1813
328
328
1801
1801
Image 148 done...
--------------------------------------------------
2592 1944
287
287
1352
1352
295
295
1261
1261
Image 149 done...
--------------------------------------------------
2736 1824
477
477
1287
1287
686
686
1409
1409
Image 150 done...
--------------------------------------------------
1725 1725
380
380
1154
1154
214
214
886
886
Image 151 done...
--------------------------------------------------
3888 2592
592
592
1921
1921
725
725
2009
2009
Image 152 done...
--------------

4096 3072
604
604
2143
2143
635
635
2027
2027
Image 215 done...
--------------------------------------------------
2592 1944
443
443
1340
1340
322
322
1270
1270
Image 216 done...
--------------------------------------------------
4752 3168
786
786
2307
2307
2257
2257
3637
3637
Image 217 done...
--------------------------------------------------
2592 1944
298
298
1441
1441
1388
1388
2441
2441
Image 218 done...
--------------------------------------------------
4928 3264
706
706
2224
2224
2491
2491
3985
3985
Image 219 done...
--------------------------------------------------
3888 2592
660
660
1761
1761
905
905
1970
1970
Image 220 done...
--------------------------------------------------
4928 3264
1062
1062
2397
2397
2520
2520
3837
3837
Image 221 done...
--------------------------------------------------
3456 2304
534
534
1725
1725
737
737
1808
1808
Image 222 done...
--------------------------------------------------
4928 3264
612
612
2580
2580
2547
2547
4173
4173
Image 223 done...
----

4752 3168
793
793
2101
2101
2377
2377
3514
3514
Image 286 done...
--------------------------------------------------
3504 2336
424
424
1900
1900
1663
1663
3022
3022
Image 287 done...
--------------------------------------------------
3456 2304
576
576
1641
1641
855
855
1914
1914
Image 288 done...
--------------------------------------------------
4928 3264
822
822
2319
2319
2650
2650
3910
3910
Image 289 done...
--------------------------------------------------
4928 3264
889
889
2401
2401
810
810
2415
2415
Image 290 done...
--------------------------------------------------
2592 1944
277
277
1510
1510
1725
1725
2736
2592
Image 291 done...
--------------------------------------------------
3696 2448
523
523
1726
1726
644
644
1691
1691
Image 292 done...
--------------------------------------------------
4752 3168
643
643
2212
2212
930
930
2424
2424
Image 293 done...
--------------------------------------------------
2560 1920
271
271
1432
1432
1313
1313
2534
2534
Image 294 done...
------

...

2592 1944
228
228
1464
1464
1138
1138
2617
2592
Image 9649 done...
--------------------------------------------------
4928 3264
967
967
2203
2203
2656
2656
3910
3910
Image 9650 done...
--------------------------------------------------
2560 1920
324
324
1362
1362
1228
1228
2335
2335
Image 9651 done...
--------------------------------------------------
3888 2592
662
662
1913
1913
1870
1870
2992
2992
Image 9652 done...
--------------------------------------------------
5184 3456
980
980
2396
2396
1305
1305
2607
2607
Image 9653 done...
--------------------------------------------------
2592 1944
350
350
1412
1412
1473
1473
2397
2397
Image 9654 done...
--------------------------------------------------
4096 3072
693
693
2100
2100
611
611
1916
1916
Image 9655 done...
--------------------------------------------------
3216 2136
462
462
1476
1476
1690
1690
2635
2635
Image 9656 done...
--------------------------------------------------
2560 1920
308
308
1466
1466
1517
1517
2591
2560
Image 9657

1792 1184
303
303
843
843
995
995
1466
1466
Image 9719 done...
--------------------------------------------------
3888 2592
49
49
1642
1642
2373
2373
3729
3729
Image 9720 done...
--------------------------------------------------
3888 2592
679
679
1867
1867
1847
1847
2960
2960
Image 9721 done...
--------------------------------------------------
2592 1944
315
315
1338
1338
1222
1222
2215
2215
Image 9722 done...
--------------------------------------------------
3456 2304
633
633
1572
1572
834
834
1701
1701
Image 9723 done...
--------------------------------------------------
2592 1944
828
828
1869
1869
256
256
1258
1258
Image 9724 done...
--------------------------------------------------
4928 3264
772
772
2308
2308
1041
1041
2418
2418
Image 9725 done...
--------------------------------------------------
3504 2336
741
741
2088
2088
179
179
1310
1310
Image 9726 done...
--------------------------------------------------
1725 1725
419
419
1199
1199
1200
1200
1863
1725
Image 9727 done...
-

4096 3072
805
805
2164
2164
2264
2264
3521
3521
Image 9789 done...
--------------------------------------------------
2592 1944
397
397
1369
1369
1270
1270
2266
2266
Image 9790 done...
--------------------------------------------------
4928 3264
813
813
2142
2142
2637
2637
3837
3837
Image 9791 done...
--------------------------------------------------
1956 1934
535
535
1444
1444
1011
1011
1770
1770
Image 9792 done...
--------------------------------------------------
2592 1944
159
159
1257
1257
254
254
1331
1331
Image 9793 done...
--------------------------------------------------
2592 1944
346
346
1339
1339
1378
1378
2242
2242
Image 9794 done...
--------------------------------------------------
2124 2056
947
947
2018
2018
185
185
1109
1109
Image 9795 done...
--------------------------------------------------
2560 1920
326
326
1451
1451
1450
1450
2512
2512
Image 9796 done...
--------------------------------------------------
1876 1876
508
508
1390
1390
87
87
870
870
Image 9797 done...

3216 2136
412
412
1513
1513
1973
1973
2867
2867
Image 9859 done...
--------------------------------------------------
3888 2592
741
741
1947
1947
942
942
1947
1947
Image 9860 done...
--------------------------------------------------
4928 3264
763
763
2314
2314
909
909
2322
2322
Image 9861 done...
--------------------------------------------------
4928 3264
763
763
2317
2317
1113
1113
2292
2292
Image 9862 done...
--------------------------------------------------
1725 1725
398
398
1280
1280
788
788
1598
1598
Image 9863 done...
--------------------------------------------------
3456 2304
494
494
1721
1721
696
696
1839
1839
Image 9864 done...
--------------------------------------------------
3888 2592
719
719
1943
1943
1899
1899
2958
2958
Image 9865 done...
--------------------------------------------------
2560 1920
164
164
1382
1382
1293
1293
2406
2406
Image 9866 done...
--------------------------------------------------
2560 1920
302
302
1334
1334
1323
1323
2376
2376
Image 9867 done.

2592 1944
149
149
1328
1328
202
202
1336
1336
Image 9930 done...
--------------------------------------------------
4928 3264
631
631
2374
2374
2686
2686
4216
4216
Image 9931 done...
--------------------------------------------------
2560 1920
345
345
1548
1548
1670
1670
2771
2560
Image 9932 done...
--------------------------------------------------
4752 3168
862
862
2572
2572
1516
1516
3034
3034
Image 9933 done...
--------------------------------------------------
5472 3648
897
897
2637
2637
1350
1350
2835
2835
Image 9934 done...
--------------------------------------------------
3456 2304
623
623
1775
1775
1685
1685
2789
2789
Image 9935 done...
--------------------------------------------------
4928 3264
770
770
2204
2204
1141
1141
2449
2449
Image 9936 done...
--------------------------------------------------
1956 1934
425
425
1388
1388
54
54
984
984
Image 9937 done...
--------------------------------------------------
3888 2592
749
749
1991
1991
671
671
1958
1958
Image 9938 done...